In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import joblib
import numpy as np


def calculate_evapotranspiration(df):
    # Constantes
    gamma = 0.065  # Constante psicrométrica (kPa/°C)
    rn_minus_g = 2.0  # Radiación neta menos flujo de calor (MJ/m²/día)
    u2 = 2.0  # Velocidad del viento (m/s)

    # Variables del dataset
    T = df['avg_air_temperature']  # Temperatura del aire (°C)
    RH = df['avg_air_humidity']  # Humedad relativa (%)

    # Cálculo de presión de vapor
    e_s = 0.6108 * np.exp((17.27 * T) / (T + 237.3))  # Saturación (kPa)
    e_a = e_s * (RH / 100)  # Presión real de vapor (kPa)

    # Pendiente de la curva de presión de vapor
    delta = (4098 * e_s) / ((T + 237.3) ** 2)  # (kPa/°C)

    # Cálculo simplificado de ET
    et = (0.408 * delta * rn_minus_g + gamma * (900 / (T + 273)) * u2 * (e_s - e_a)) / \
         (delta + gamma * (1 + 0.34 * u2))

    return et


def predict_evapotranspiration(csv_path: str, model_output: str):
    print('Cargando datos desde', csv_path)

    # Cargar y preparar los datos
    df = pd.read_csv(csv_path)
    df['hour'] = pd.to_datetime(df['hour'], errors="coerce")
    df['evapotranspiration_rate'] = calculate_evapotranspiration(df)
    df = df.dropna()

    # Seleccionar variables relevantes
    X = df[['avg_air_temperature', 'avg_air_humidity', 'avg_soil_humidity']]
    y = df['evapotranspiration_rate']  # Variable objetivo: tasa de ET en mm/hora

    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Crear el modelo de la red neuronal
    model = Sequential([
        Dense(64, input_dim=X_train.shape[1], activation='relu'),  # Capa oculta 1
        Dropout(0.2),  # Regularización para evitar sobreajuste
        Dense(32, activation='relu'),  # Capa oculta 2
        Dropout(0.2),
        Dense(1, activation='linear')  # Capa de salida (predicción continua)
    ])

    # Compilar el modelo
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

    # Entrenar el modelo
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=50,
        batch_size=16,
        verbose=1
    )

    # Evaluar el modelo
    loss, mae = model.evaluate(X_test, y_test, verbose=0)
    print(f"Mean Absolute Error (MAE): {mae:.2f}")

    # Hacer predicciones
    y_pred = model.predict(X_test)

    # Comparar predicciones con los valores reales
    results = pd.DataFrame({
        'Actual': y_test.values,
        'Predicted': y_pred.flatten()
    })
    print(results.head())

    # Guardar el modelo
    joblib.dump(model, model_output)
    print(f"Modelo guardado en {model_output}")


Cargando datos desde csv/per_hour.csv
Epoch 1/50


C:\Users\gerojasf\Desktop\projects\capstone\farming-smart-analytics\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 88.8338 - mae: 7.6256 - val_loss: 9.6627 - val_mae: 2.7924
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 57.2581 - mae: 6.4213 - val_loss: 7.5505 - val_mae: 2.3347
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 49.4325 - mae: 5.3093 - val_loss: 3.7438 - val_mae: 1.7356
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 47.7934 - mae: 5.4788 - val_loss: 2.1806 - val_mae: 1.3784
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 35.9762 - mae: 4.7987 - val_loss: 2.2498 - val_mae: 1.2387
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 30.0057 - mae: 4.6122 - val_loss: 3.7917 - val_mae: 1.7779
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 40.5821 - mae: 5.0099 - val_loss: 2.9303 - val_mae: 1.5251
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 36.8346 - mae: 4.7738 - val_loss: 2.6842 - val_mae: 1.5028
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 24.4933 - mae: 3.8241 - va